# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

from pathlib import Path

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


D:\Anaconda3\envs\nlp-2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
w2v_folder = Path()
raw_data_folder = w2v_folder / 'data-raw'
models_folder = w2v_folder / 'models'
path_to_data_file = raw_data_folder /'reviews_data.txt.gz'

create_new_model = False

In [3]:
# data_file=str(path_to_data_file.absolute())

In [4]:
with gzip.open (path_to_data_file, 'rb') as f:
# with open (data_file, 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break

b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [5]:
def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info(f"reading file {input_file}...this may take a while")
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info (f"read {i} reviews")
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess(line)


if create_new_model:
#   read the tokenized reviews into a list
#   each review item becomes a series of words
#   so this becomes a list of lists
    documents = list (read_input (data_file))
    logging.info("Done reading data files")    
else: 
    logging.info('Not reading text files as existing model will be loaded')

2019-05-27 19:38:16,941 : INFO : Not reading text files as existing model will be loaded


## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [6]:
vector_size = 152

path_to_save_model = models_folder / f'hotel-reviews-{vector_size}-01.model'
path_to_load_model = models_folder / f'hotel-reviews-{vector_size}-01.model'

str_to_save_model = str(path_to_save_model.absolute())
str_to_load_model = str(path_to_load_model.absolute())

if create_new_model:
    model = gensim.models.Word2Vec (documents, size=vector_size, window=10, min_count=2, workers=10)
    model.train(documents,total_examples=len(documents),epochs=10)
    model.save(str_to_save_model)
else:
    model = gensim.models.Word2Vec.load(str_to_load_model)

2019-05-27 19:38:33,552 : INFO : loading Word2Vec object from D:\PyProjects\_references_nb\10_ml-stats-applications\nlp-00-nlp-in-practive\1.00_my_word2vec\models\hotel-reviews-152-01.model
2019-05-27 19:38:33,724 : INFO : loading wv recursively from D:\PyProjects\_references_nb\10_ml-stats-applications\nlp-00-nlp-in-practive\1.00_my_word2vec\models\hotel-reviews-152-01.model.wv.* with mmap=None
2019-05-27 19:38:33,725 : INFO : loading vectors from D:\PyProjects\_references_nb\10_ml-stats-applications\nlp-00-nlp-in-practive\1.00_my_word2vec\models\hotel-reviews-152-01.model.wv.vectors.npy with mmap=None
2019-05-27 19:38:33,754 : INFO : setting ignored attribute vectors_norm to None
2019-05-27 19:38:33,755 : INFO : loading vocabulary recursively from D:\PyProjects\_references_nb\10_ml-stats-applications\nlp-00-nlp-in-practive\1.00_my_word2vec\models\hotel-reviews-152-01.model.vocabulary.* with mmap=None
2019-05-27 19:38:33,755 : INFO : loading trainables recursively from D:\PyProjects\_

Save the trained model

Word2Vec has several components: the full model (just saved) but also the **KeyedVectors** accessible by `model.wv` and that is the matrix of all the trained word vectors (one set of weight for each word in vocabulary. In this case, this is 152 weight as the model was trained with `size=152`. `model.wv` is more compact than the full model 

In [7]:
word = 'dirty'
keyedvectors = model.wv[word]
print(f'Each word such as <{word}> is represented by {len(keyedvectors)} weights. Example:')
display(keyedvectors)

Each word such as <dirty> is represented by 152 weights. Example:


array([ 0.93197924,  0.5105132 ,  2.2606149 , -3.1462505 , -0.8515029 ,
        4.1018205 ,  1.910454  , -0.5651825 ,  0.12677635,  3.6665273 ,
       -0.7763401 , -2.694595  ,  1.9538004 ,  2.5295408 , -1.0050497 ,
       -0.60198313,  0.7992244 , -0.57420427, -3.8793583 ,  0.7293769 ,
        3.5143297 , -5.7852254 ,  0.04295096,  2.815917  , -0.84027684,
       -0.8289401 , -2.201959  ,  1.2326481 , -4.1024055 , -1.531972  ,
        6.165753  ,  0.34523195,  0.7494303 ,  3.4426212 , -4.784218  ,
        0.10629007,  0.35603303, -2.3511071 ,  1.8411856 , -4.230956  ,
        0.14763859, -2.1860747 ,  1.1594225 , -3.1501563 ,  0.8527438 ,
        0.56384283, -4.7642007 , -2.5231395 ,  1.3898998 , -0.39603552,
       -3.28744   , -0.38484225,  4.3891683 ,  1.1029959 ,  0.9481255 ,
       -2.959008  ,  0.6458025 ,  1.7236764 , -0.4991107 , -2.8156686 ,
       -1.9842317 ,  0.8331184 , -0.35104042, -2.0710475 , -1.9407701 ,
       -1.0832196 ,  1.9560636 ,  0.8210384 ,  1.3318613 ,  2.43

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [29]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

2019-05-11 10:54:12,597 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8638341426849365),
 ('unclean', 0.7788464426994324),
 ('stained', 0.7784373760223389),
 ('grubby', 0.7623147964477539),
 ('dusty', 0.760489284992218),
 ('smelly', 0.7517919540405273),
 ('dingy', 0.7328144311904907),
 ('disgusting', 0.721342146396637),
 ('mouldy', 0.7155332565307617),
 ('gross', 0.7071250081062317)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [8]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1, topn=6)

2019-05-27 19:39:19,291 : INFO : precomputing L2-norms of word weight vectors


[('courteous', 0.9179747104644775),
 ('friendly', 0.8312869071960449),
 ('cordial', 0.8024935722351074),
 ('professional', 0.7896497249603271),
 ('curteous', 0.7840185761451721),
 ('attentive', 0.7761869430541992)]

In [9]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('spain', 0.6637556552886963),
 ('canada', 0.6535975933074951),
 ('germany', 0.6516804099082947),
 ('barcelona', 0.6219171285629272),
 ('rome', 0.6058337688446045),
 ('england', 0.6040583848953247)]

In [10]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.8222155570983887),
 ('amazed', 0.7932467460632324),
 ('astonished', 0.7825183868408203),
 ('appalled', 0.7789210081100464),
 ('dismayed', 0.7773011922836304),
 ('stunned', 0.7551657557487488)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [11]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('mattress', 0.7097318768501282),
 ('duvet', 0.706474244594574),
 ('matress', 0.6960698366165161),
 ('quilt', 0.6954215168952942),
 ('blanket', 0.6837774515151978),
 ('pillowcase', 0.644959568977356),
 ('foam', 0.6408494114875793),
 ('sheets', 0.6368979811668396),
 ('pillows', 0.6310677528381348),
 ('pillowcases', 0.6079158782958984)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [12]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.7625367507867546

In [13]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0

In [14]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.2685083373940712

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [15]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [16]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
